In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import cv2
import os
import numpy as np
from easydict import EasyDict as edict
import tensorflow.contrib.slim as slim


from google.colab import drive
drive.mount('/content/drive')


__C = edict()
cfg = __C
__C.DIR = '/content/drive/My Drive/VSimplE/'
__C.POOLING_SIZE = 7
__C.VRD_BATCH_NUM = 30
__C.VRD_NUM_CLASS = 101
__C.VRD_NUM_RELA = 70
__C.VRD_LR_INIT = 0.00001
__C.VRD_TRAIN_ROUND = 20
__C.VRD_BATCH_NUM_RELA = 50
__C.VRD_AU_PAIR = 5
__C.VRD_IOU_TRAIN = 0.5
__C.VRD_IOU_TEST = 0.5
__C.IM_SIZE = 600
__C.IM_MAX_SIZE = 1000
__C.TRAIN = edict()
__C.RESNET = edict()
__C.RESNET.FIXED_BLOCKS = 1
__C.VTR = edict()

print ("tf  :       ",tf.__version__)
print ("cv2 :       ",cv2.__version__)
print ("np  :       ",np.__version__)


def generate_batch(N_total, N_each):
    num_batch = np.int32(N_total / N_each)
    if N_total % N_each == 0:
        index_box = range(N_total)
    else:
        index_box = np.empty(shape=[N_each * (num_batch + 1)], dtype=np.int32)
        index_box[0:N_total] = range(N_total)
        N_rest = N_each * (num_batch + 1) - N_total
        index_box[N_total:] = np.random.randint(0, N_total, N_rest)
    return index_box

def read_roidb(roidb_path):
    roidb_file = np.load(roidb_path, allow_pickle=True)
    key = list(roidb_file.keys())[0]
    roidb_temp = roidb_file[key]
    roidb = roidb_temp[()]
    return roidb
#_______________________________________________________________________________
co_occurency_tensor = read_roidb(cfg.DIR + 'input/co_occurency_tensor.npz')
P_op = read_roidb(cfg.DIR + 'input/P_op.npz')
P_sp = read_roidb(cfg.DIR + 'input/P_sp.npz')
P_p = read_roidb(cfg.DIR  + 'input/P_p.npz')

def im_preprocess(image_path):
    image = cv2.imread(image_path)
    im_orig = image.astype(np.float32, copy=True)
    im_orig -= np.array([[[102.9801, 115.9465, 122.7717]]])
    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])
    target_size = cfg.IM_SIZE
    max_size = cfg.IM_MAX_SIZE
    im_scale = float(target_size) / float(im_size_min)
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
    im_shape_new = np.shape(im)
    im_use = np.zeros([1, im_shape_new[0], im_shape_new[1], im_shape_new[2]])
    im_use[0, :, :, :] = im
    return im_use, im_scale    

########################## spatial_vector     ###################################
def spatial_vector(sbbox, obbox, N_each_batch):
    spatial_vector = np.zeros([N_each_batch, 26])
    ubbox = generate_phrase_box(sbbox, obbox)
    for i in range(N_each_batch):
        x_s = (sbbox[i][0] + sbbox[i][2]) / 2.0
        y_s = (sbbox[i][1] + sbbox[i][3]) / 2.0
        w_s = sbbox[i][2] - sbbox[i][0]
        h_s = sbbox[i][3] - sbbox[i][1]

        x_o = (obbox[i][0] + obbox[i][2]) / 2.0
        y_o = (obbox[i][1] + obbox[i][3]) / 2.0
        w_o = obbox[i][2] - obbox[i][0]
        h_o = obbox[i][3] - obbox[i][1]

        x_u = (ubbox[i][0] + ubbox[i][2]) / 2.0
        y_u = (ubbox[i][1] + ubbox[i][3]) / 2.0
        w_u = ubbox[i][2] - ubbox[i][0]
        h_u = ubbox[i][3] - ubbox[i][1]
        spatial_vector[i] = [x_s, y_s, w_s / w_o, h_s / h_o, ((w_s * h_s) / (w_o * h_o)),
                            (x_s - x_o) / w_o, (y_s - y_o) / h_o, w_s / 600, h_s / 600,
                            np.log(w_s / w_o), np.log(h_s / h_o),

                            x_u, y_u, w_s / w_u, h_s / h_u, (w_s * h_s) / (w_u * h_u),
                            (x_s - x_u) / w_u, (y_s - y_u) / h_u, w_u / 600, h_u / 600,
                            np.log(w_s / w_u), np.log(h_s / h_u),

                            (sbbox[i][0] - ubbox[i][0]) / w_u, (sbbox[i][1] - ubbox[i][1]) / h_u,
                            (sbbox[i][2] - ubbox[i][2]) / w_u, (sbbox[i][3] - ubbox[i][3]) / h_u
                             ]
    return spatial_vector
#####################################################     IoU      ###################################################
def compute_iou_batch(sbbox, obbox, N_each_batch):
    iou = np.zeros([N_each_batch, 4])
    for i in range(N_each_batch):
        xA = max(sbbox[i][0], obbox[i][0])
        yA = max(sbbox[i][1], obbox[i][1])
        xB = min(sbbox[i][2], obbox[i][2])
        yB = min(sbbox[i][3], obbox[i][3])
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        sbbox_Area = (sbbox[i][2] - sbbox[i][0] + 1) * (sbbox[i][3] - sbbox[i][1] + 1)
        obbox_Area = (obbox[i][2] - obbox[i][0] + 1) * (obbox[i][3] - obbox[i][1] + 1)
        iou[i] = interArea / float(sbbox_Area + obbox_Area - interArea)
    return iou

#####################	co_occurency	##############################
def co_occurency(sub_gt,ob_gt,N_each_batch):
	co_occurency_list = np.zeros((30,70))
	for i in range(N_each_batch):
		sub_idx = sub_gt[i]
		ob_idx = ob_gt[i]
		co_occurency_temp = np.zeros((70))
		for j in range (70):
			PMI = np.log10(((co_occurency_tensor[sub_idx][ob_idx][j]) * P_p[j]) / (P_sp[sub_idx][j] * P_op[ob_idx][j]))
			if PMI > 0 :
				co_occurency_temp[j] = PMI
			else:
				co_occurency_temp[j] = 0	
		co_occurency_list[i] = co_occurency_temp
	return co_occurency_list
##################################################
def get_blob_pred(roidb_use, im_scale, index_sp, N_each_batch, batch_id):
    blob = {}
    sub_box = roidb_use['sub_box_gt']*im_scale
    obj_box = roidb_use['obj_box_gt']*im_scale
    rela = np.int32(roidb_use['rela_gt'])
    index = roidb_use['index_pred']
    sub_gt = np.int32(roidb_use['sub_gt'])
    ob_gt = np.int32(roidb_use['obj_gt'])
    index_use = index[batch_id*N_each_batch: (batch_id+1)*N_each_batch]
    sub_gt_use = sub_gt[index_use]
    ob_gt_use = ob_gt[index_use]
    sub_box_use = sub_box[index_use,:]
    obj_box_use = obj_box[index_use,:]
    rela_use = rela[index_use]
    co_occurency_log = co_occurency(sub_gt_use, ob_gt_use, N_each_batch)
    blob['sub_label'] = sub_gt_use
    blob['obj_label'] = ob_gt_use
    blob['sub_box'] = sub_box_use
    blob['obj_box'] = obj_box_use
    blob['rela'] = rela_use
    blob['iou'] = compute_iou_batch(sub_box_use , obj_box_use , N_each_batch)
    blob['union_box'] = generate_phrase_box(sub_box_use , obj_box_use)
    blob['sub_spatial_vector'] = spatial_vector(sub_box_use, obj_box_use ,N_each_batch)
    blob['ob_spatial_vector'] = spatial_vector(obj_box_use, sub_box_use ,N_each_batch)
    blob['co_occurency_tensor'] = co_occurency_log
    return blob
############################################################################################################
def generate_phrase_box(sbox, obox):
    N_box = len(sbox)
    phrase = np.zeros([N_box, 4])
    for i in range(N_box):
        phrase[i, 0] = min(sbox[i, 0], obox[i, 0])
        phrase[i, 1] = min(sbox[i, 1], obox[i, 1])
        phrase[i, 2] = max(sbox[i, 2], obox[i, 2])
        phrase[i, 3] = max(sbox[i, 2], obox[i, 3])
    return phrase
# ________________________________________________________________________________________________________#
class VSimplE(object):
    def __init__(self):
        self.predictions = {}
        self.losses = {}
        self.layers = {}
        self.feat_stride = [16, ]
        self.scope = 'vgg_16'

    def create_graph(self, N_each_batch, index_sp, index_cls, num_classes, num_predicates):
        self.image      = tf.placeholder(tf.float32, shape=[1, None, None, 3])
        self.sub_label  = tf.placeholder(tf.int32, shape=[N_each_batch, ])
        self.obj_label  = tf.placeholder(tf.int32, shape=[N_each_batch, ])        
        self.sbox       = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
        self.obox       = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
        self.iou        = tf.placeholder(tf.float32 , shape=[N_each_batch , 4] )
        self.union_box  = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
        self.sub_sp     = tf.placeholder(tf.float32, shape=[N_each_batch, 26])
        self.ob_sp      = tf.placeholder(tf.float32, shape=[N_each_batch, 26])
        self.co_occurency_log = tf.placeholder(tf.float32 , shape=[N_each_batch , 70] )
        self.rela_label = tf.placeholder(tf.int32, shape=[N_each_batch, ])
        self.keep_prob  = tf.placeholder(tf.float32)

        self.sub_obj_p = tf.placeholder(tf.float32, shape=[None, num_predicates])
        self.object_word = np.load(cfg.DIR +'/input/oList_word_embedding.npy')
        self.robject = np.load(cfg.DIR +'/input/rList_word_embedding.npy')
        conf = np.load(cfg.DIR +'/input/language_inter.npz')
        self.sub_obj_pred = conf['sub_obj']
        self.sub_pred = conf['sub']
        self.obj_pred = conf['obj']

        self.index_sp = index_sp
        self.index_cls = index_cls
        self.num_classes = num_classes
        self.num_predicates = num_predicates
        self.N_each_batch = N_each_batch

        self.build_dete_network()
        self.setup_ent_emb()
        self.build_rd_network()
        self.add_rd_loss()

#################################__________VGG 16____________################################################

    def build_dete_network(self, is_training=True):
        img= self.image
        net_conv = self.image_to_head(is_training)
        sub_pool5 = self.crop_pool_layer(net_conv, self.sbox, "sub_pool5")
        ob_pool5 = self.crop_pool_layer(net_conv, self.obox, "ob_pool5")
        sub_fc7 = self.head_to_tail(sub_pool5, is_training, reuse=False)
        ob_fc7 = self.head_to_tail(ob_pool5, is_training, reuse=True)
        with tf.variable_scope(self.scope, self.scope):
            sub_cls_prob, sub_cls_pred = self.region_classification(sub_fc7, is_training, reuse=False)
        with tf.variable_scope(self.scope, self.scope):
            ob_cls_prob, ob_cls_pred = self.region_classification(ob_fc7, is_training, reuse=True)

        self.predictions['sub_cls_prob'] = sub_cls_prob
        self.predictions['sub_cls_pred'] = sub_cls_pred
        self.predictions['ob_cls_prob'] = ob_cls_prob
        self.predictions['ob_cls_pred'] = ob_cls_pred
        # self.layers['sub_pool5'] = sub_pool5
        # self.layers['ob_pool5'] = ob_pool5
        self.layers['sub_fc7'] = sub_fc7
        self.layers['ob_fc7'] = ob_fc7

    def image_to_head(self, is_training, reuse=False):
        with tf.variable_scope(self.scope, self.scope, reuse=reuse):
            print("Image_shape :  ",self.image)
            net = slim.repeat(self.image, 2, slim.conv2d, 64, [3, 3],trainable=is_training, scope='conv1')
            net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3],trainable=is_training, scope='conv2')
            net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool2')
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3],trainable=is_training, scope='conv3')
            net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool3')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3],trainable=is_training, scope='conv4')
            net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool4')
            net_conv = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], trainable=is_training, scope='conv5')
            self.layers['head'] = net_conv
            return net_conv

    def head_to_tail(self, pool5, is_training, reuse=False):
        with tf.variable_scope(self.scope, self.scope, reuse=reuse):
            pool5_flat = slim.flatten(pool5, scope='flatten')
            fc6 = slim.fully_connected(pool5_flat, 4096, scope='fc6')
            fc6 = slim.dropout(fc6, keep_prob=self.keep_prob, is_training=True,scope='dropout6')
            fc7 = slim.fully_connected(fc6, 4096, scope='fc7')
            fc7 = slim.dropout(fc7, keep_prob=self.keep_prob, is_training=True,scope='dropout7')
            return fc7

    def crop_pool_layer(self, bottom, rois, name):
        with tf.variable_scope(name) as scope:
            n = tf.to_int32(rois.shape[0])
            batch_ids = tf.zeros([n, ], dtype=tf.int32)
            bottom_shape = tf.shape(bottom)
            height = (tf.to_float(bottom_shape[1]) - 1.) * np.float32(self.feat_stride[0])
            width = (tf.to_float(bottom_shape[2]) - 1.) * np.float32(self.feat_stride[0])
            x1 = tf.slice(rois, [0, 0], [-1, 1], name="x1") / width
            y1 = tf.slice(rois, [0, 1], [-1, 1], name="y1") / height
            x2 = tf.slice(rois, [0, 2], [-1, 1], name="x2") / width
            y2 = tf.slice(rois, [0, 3], [-1, 1], name="y2") / height
            bboxes = tf.stop_gradient(tf.concat([y1, x1, y2, x2], 1))
            crops = tf.image.crop_and_resize(bottom, bboxes, tf.to_int32(batch_ids),[cfg.POOLING_SIZE * 2, cfg.POOLING_SIZE * 2], method='bilinear',name="crops")
            pooling = max_pool(crops, 2, 2, 2, 2, name="max_pooling")
        return pooling

    def region_classification(self, fc7, is_training, reuse=False):
        cls_score = slim.fully_connected(fc7, self.num_classes,activation_fn=None, scope='cls_score', reuse=reuse)
        print("cls_score's shape: {0}".format(cls_score.get_shape()))
        cls_prob = tf.nn.softmax(cls_score, name="cls_prob")
        cls_pred = tf.argmax(cls_score, axis=1, name="cls_pred")
        return cls_prob, cls_pred
        
############################################      Relationship detection          ###########################################################
    def setup_ent_emb(self):
        self.head = tf.get_variable(name="RD_head_emb",initializer=tf.random_uniform(shape=[30, 2000], minval=0, maxval=1999,dtype=tf.int32))
        self.tail = tf.get_variable(name="RD_tail_emb",initializer=tf.random_uniform(shape=[30, 2000], minval=0, maxval=1999,dtype=tf.int32))

    def Bi_GRU(self, inputs, name):
        with tf.variable_scope(name):
            fw_gru_cell = tf.contrib.rnn.GRUCell(1000)
            bw_gru_cell = tf.contrib.rnn.GRUCell(1000)
            fw_gru_cell = tf.contrib.rnn.DropoutWrapper(fw_gru_cell, output_keep_prob=0.8)
            bw_gru_cell = tf.contrib.rnn.DropoutWrapper(bw_gru_cell, output_keep_prob=0.8)
            (fw_outputs, bw_outputs),_ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_gru_cell,cell_bw=bw_gru_cell,inputs=inputs,dtype=tf.float32)
            outputs = tf.concat((fw_outputs, bw_outputs), 2)
            outputs = tf.reshape(outputs,[30,2000])
        return outputs

    def max_norm_regularizer(self, threshold, axes=1, name="max_norm", collection="max_norm"):
        def max_norm(weights):
            clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
            clip_weights = tf.assign(weights, clipped, name=name)
            tf.add_to_collection(collection, clip_weights)
            return None
        return max_norm

    def build_rd_network(self):
        #_________________________Sub_feature
        sub_cls_prob = self.predictions['sub_cls_prob']
        sub_cls_pred = self.predictions['sub_cls_pred']
        sub_fc = self.layers['sub_fc7'] 
        sbox = self.sbox
        sub_sp = self.sub_sp
        #_________________________Obj_feature
        ob_cls_prob = self.predictions['ob_cls_prob']
        ob_cls_pred = self.predictions['ob_cls_pred']
        ob_fc = self.layers['ob_fc7']
        obox = self.obox
        ob_sp = self.ob_sp
        #_____________________________Spatial
        co_occur = self.co_occurency_log
        union_box = self.union_box
        iou = self.iou
        sub_sp_info = tf.concat([iou, sbox, sub_sp, union_box], axis=1)
        ob_sp_info  = tf.concat([iou, obox, ob_sp , union_box], axis=1)
        #________________________________Word2Vec
        vector_dic = tf.Variable(self.object_word, trainable=False, name='VD_vo')
        sub_onehot = tf.matmul(tf.one_hot(self.sub_label, self.num_classes - 1), vector_dic)
        obj_onehot = tf.matmul(tf.one_hot(self.obj_label, self.num_classes - 1), vector_dic)
#______________________________________________________4. Balanced Size of extracted features
        sub_sp_info  = slim.fully_connected(sub_sp_info,  1000, activation_fn=tf.nn.relu, scope='RD_subinfo_fc',     weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        sub_cls_prob = slim.fully_connected(sub_cls_prob, 500, activation_fn=tf.nn.relu, scope='RD_subclsprob_fc',  weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        sub_fc       = slim.fully_connected(sub_fc,      3000, activation_fn=tf.nn.relu, scope='RD_subfc_fc',       weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        label_s      = slim.fully_connected(sub_onehot , 1000, activation_fn=tf.nn.relu, scope='RD_ls1',            weights_regularizer=self.max_norm_regularizer(threshold=0.85))

        ob_sp_info   = slim.fully_connected(ob_sp_info,   1000, activation_fn=tf.nn.relu, scope='RD_obinfo_fc',      weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        ob_cls_prob  = slim.fully_connected(ob_cls_prob,  500, activation_fn=tf.nn.relu, scope='RD_obclsprob_fc',   weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        ob_fc        = slim.fully_connected(ob_fc,       3000, activation_fn=tf.nn.relu, scope='RD_obfc_fc',        weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        label_o      = slim.fully_connected(obj_onehot , 1000, activation_fn=tf.nn.relu, scope='RD_lo1',            weights_regularizer=self.max_norm_regularizer(threshold=0.85))       
#______________________Drop out
        sub_sp_info  = tf.layers.dropout(sub_sp_info , rate=0.2, training=True)
        sub_cls_prob = tf.layers.dropout(sub_cls_prob, rate=0.2, training=True)
        sub_fc       = tf.layers.dropout(sub_fc		 , rate=0.2, training=True)
        label_s 	 = tf.layers.dropout(label_s	 , rate=0.2, training=True)

        ob_sp_info  = tf.layers.dropout(ob_sp_info	, rate=0.2, training=True)
        ob_cls_prob = tf.layers.dropout(ob_cls_prob	, rate=0.2, training=True)
        ob_fc 		= tf.layers.dropout(ob_fc		, rate=0.2, training=True)
        label_s 	= tf.layers.dropout(label_s		, rate=0.2, training=True)

        if self.index_sp:
            sub_fc = tf.concat([sub_fc,label_s, sub_sp_info, sub_cls_prob], axis=1)
            ob_fc  = tf.concat([ob_fc ,label_o, ob_sp_info , ob_cls_prob ], axis=1)
        if self.index_cls:
            sub_fc = tf.concat([sub_fc,label_s, sub_sp_info, sub_cls_prob], axis=1)
            ob_fc  = tf.concat([ob_fc ,label_o , ob_sp_info, ob_cls_prob ], axis=1)
        #________________________________________Embedding space
        sub_fc = slim.fully_connected(sub_fc, 2000, activation_fn=tf.nn.relu, scope='RD_sub_fc',weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        ob_fc  = slim.fully_connected(ob_fc , 2000, activation_fn=tf.nn.relu, scope='RD_ob_fc' ,weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        sub_fc = tf.layers.dropout(sub_fc, rate=0.2, training=True)
        ob_fc  = tf.layers.dropout(ob_fc , rate=0.2, training=True)
#__________________________________________________________________Sampling Head and tail       
        s = tf.shape(self.head)
        batch_idx = tf.tile(tf.expand_dims(tf.range(s[0]), 1), [1, s[1]])
        sub_head = tf.gather_nd(params=sub_fc, indices=tf.stack([batch_idx, self.head], axis=-1), name='RD_sub_head')
        sub_tail = tf.gather_nd(params=sub_fc, indices=tf.stack([batch_idx, self.tail], axis=-1), name='RD_sub_tail')
        ob_head  = tf.gather_nd(params=ob_fc,  indices=tf.stack([batch_idx, self.head], axis=-1), name='RD_ob_head')
        ob_tail  = tf.gather_nd(params=ob_fc,  indices=tf.stack([batch_idx, self.tail], axis=-1), name='RD_ob_tail')

        sub_head = slim.fully_connected(sub_head, 2000, activation_fn=tf.nn.relu, scope='RD_subhead_fc',weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        ob_head  = slim.fully_connected(ob_head , 2000, activation_fn=tf.nn.relu, scope='RD_obhead_fc' ,weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        sub_tail = slim.fully_connected(sub_tail, 2000, activation_fn=tf.nn.relu, scope='RD_subtail_fc',weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        ob_tail  = slim.fully_connected(ob_tail , 2000, activation_fn=tf.nn.relu, scope='RD_obtail_fc' ,weights_regularizer=self.max_norm_regularizer(threshold=0.85))
        
        sub_head = tf.layers.dropout(sub_head, rate=0.2, training=True)
        ob_head  = tf.layers.dropout(ob_head , rate=0.2, training=True)  
        sub_tail = tf.layers.dropout(sub_tail, rate=0.2, training=True)
        ob_tail  = tf.layers.dropout(ob_tail , rate=0.2, training=True)      
        #___________________________________1.VSimplE 
        vsimple_vec = ((ob_tail - sub_head) + (sub_tail - ob_head)) / 2.0
#_________________________________________________________________________      Internal Language
        # sub_obj_p = slim.fully_connected(self.sub_obj_p, 500, activation_fn=tf.nn.relu, scope='RD_l1', reuse=False)
        # vsimple_vec = tf.concat([vsimple_vec, sub_obj_p], axis=1)
        vsimple_vec = slim.fully_connected(vsimple_vec, 2000, activation_fn=tf.nn.relu ,scope='RD_vsimple',weights_regularizer=self.max_norm_regularizer(threshold=0.9))
        vsimple_vec = tf.layers.dropout(vsimple_vec, rate=0.2, training=True)        
#___________________________________________________________________6.Bi_GRU
        gru_vec = tf.concat([sub_fc, vsimple_vec, ob_fc], axis=1)
        gru_vec = slim.fully_connected(gru_vec, 2000, activation_fn=tf.nn.relu, scope='RD_fc_GRU',weights_regularizer=self.max_norm_regularizer(threshold=0.9))
        gru_vec = tf.layers.dropout(gru_vec, rate=0.2, training=True)
        gru_vec = tf.expand_dims(gru_vec, 0)
        gru_vec = self.Bi_GRU(gru_vec, "RD_GRU")

        PMI_score   = slim.fully_connected(co_occur   , 500, activation_fn=tf.nn.relu, scope='RD_fc1',weights_regularizer=self.max_norm_regularizer(threshold=0.9))
        vsimple_vec = slim.fully_connected(vsimple_vec, 500, activation_fn=tf.nn.relu, scope='RD_fc2',weights_regularizer=self.max_norm_regularizer(threshold=0.9))
        gru_vec     = slim.fully_connected(gru_vec    , 500, activation_fn=tf.nn.relu, scope='RD_fc3',weights_regularizer=self.max_norm_regularizer(threshold=0.9))
        #______________________Drop out
        vsimple_vec = tf.layers.dropout(vsimple_vec , rate=0.2, training=True)
        PMI_score   = tf.layers.dropout(PMI_score   , rate=0.2, training=True)
        gru_vec     = tf.layers.dropout(gru_vec     , rate=0.2, training=True)
 # _________________________________________________________________13.Score of VsimplE & GRU
        vsimple_score = slim.fully_connected(vsimple_vec, self.num_predicates, activation_fn=None, scope='RD_vsimple_score')
        gru_score     = slim.fully_connected(gru_vec    , self.num_predicates, activation_fn=None, scope='RD_gru_score')
        self.layers['vsimple_score'] = vsimple_score
        self.layers['gru_score']     = gru_score   
#___________________________________________________________________7. Feature Fusion
        rela_vec    = tf.concat([PMI_score, vsimple_vec, gru_vec], axis=1)
        rela_vec    = slim.fully_connected(rela_vec, 500, activation_fn=tf.nn.relu, scope='RD_fc4',weights_regularizer=self.max_norm_regularizer(threshold=1.0))
        rela_score  = slim.fully_connected(rela_vec, self.num_predicates, activation_fn=None, scope='RD_fc5')     
        self.layers['rela_score'] = rela_score
################################________________MULTI Class CALSSIFICATION___________________#############################################
        rela_prob = tf.nn.softmax(rela_score)      
        self.layers['rela_prob'] = rela_prob
##############################__________________MULTI LABEL CALSSIFICATION___________________#############################################
#         self.predictions['rela_pred_all'] = tf.concat([tf.nn.sigmoid(rela_score)], axis = 1)
#         rela_label 	= tf.one_hot( self.rela_label, self.num_predicates)
#         self.layers['rd_loss'] = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits( labels=rela_label, logits=rela_score))

    def add_rd_loss(self):
################################________________MULTI Class CALSSIFICATION___________________#############################################
        gru_score   = self.layers['gru_score']
        rela_score  = self.layers['rela_score']
        rela_label  = self.rela_label
        vsimple_score = self.layers['vsimple_score']
        #_____________________________________________1.vsimple_Loss
        vsimple_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=rela_label, logits=vsimple_score))
        # ____________________________________________2.gru_Loss
        gru_loss     = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=rela_label, logits=gru_score))
        #_____________________________________________3.Total_Loss
        rd_loss      = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=rela_label, logits=rela_score))
#_____________________________________________15.L2_regularizeation__________________________________________________________________________________________      
        # regularizer = tf.contrib.layers.l2_regularizer(0.0001)
        # we = tf.trainable_variables()
        # RD_var = [var for var in we if 'RD' in var.name]
        # rd_loss = tf.contrib.layers.apply_regularization(regularizer, weights_list=RD_var) + rd_loss
#_______________________________________________________________________________________________________________________________________      
        acc_each = tf.nn.in_top_k(rela_score, rela_label, 1)
        self.losses['gru_loss']= gru_loss
        self.losses['rd_loss'] = rd_loss
        self.losses['acc_each']= acc_each
        self.losses['acc']     = tf.reduce_mean(tf.cast(acc_each, tf.float32))
        self.losses['vsimple_loss']  = vsimple_loss

##############################_______________MULTI LABEL CALSSIFICATION___________________#############################################
#         gru_score = self.layers['gru_score']
#         vsimple_score = self.layers['vsimple_score']
#         rela_score = tf.nn.sigmoid(self.layers['rela_score'])
#         # regularizer = tf.contrib.layers.l2_regularizer(0.0001)
#         # we = tf.trainable_variables()
#         # RD_var = [var for var in we if 'RD' in var.name]
#         rela_label = tf.one_hot(self.rela_label, self.num_predicates)
#         # rd_loss = tf.contrib.layers.apply_regularization(regularizer, weights_list=RD_var) + self.layers['rd_loss']
#         rd_loss = self.layers['rd_loss']
#         prediction = tf.one_hot(tf.argmax(rela_score, 1), self.num_predicates)
#         correct_prediction = tf.not_equal(tf.argmax(tf.multiply(prediction, tf.cast(rela_label, tf.float32)), 1), False)
#         rela_pred = tf.argmax(rela_score, 1)
#         rela_max_prob = tf.reduce_max(rela_score, 1)
#         # _____________________________________________2.vsimple_Loss
#         #vsimple_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=rela_label, logits=vsimple_score))
#         vsimple_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits( labels=rela_label, logits=vsimple_score))
#         # ____________________________________________3.gru_Loss
#         #gru_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=rela_label, logits=gru_score))
#         gru_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits( labels=rela_label, logits=gru_score))
#         # _____________________________________________4.Total_Loss
#         #acc_each = tf.nn.in_top_k(rela_score, rela_label, 1)
#         self.losses['gru_loss'] = gru_loss
#         self.losses['vsimple_loss'] = vsimple_loss
#         self.losses['rd_loss'] = rd_loss
#         #self.losses['acc_each'] = acc_each
#         self.losses['rd_loss'] = rd_loss
#         self.predictions['rela_pred'] = rela_pred
#         self.predictions['rela_max_prob'] = rela_max_prob
#         self.losses['acc'] = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def train_predicate(self, sess, roidb_use, RD_train):
        im, im_scale = im_preprocess(roidb_use['image'])
        batch_num = len(roidb_use['index_pred'])/self.N_each_batch;print("##########@@@@@@@@@@@@###########	batch_num	: ",batch_num)
        gru_loss = 0.0
        vsimple_loss = 0.0        
        RD_loss = 0.0
        acc = 0.0
        for batch_id in range(np.int32(batch_num)):
            blob = get_blob_pred(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
            feed_dict = {self.image: im, self.co_occurency_log : blob['co_occurency_tensor'],
						 self.sbox: blob['sub_box'], self.obox: blob['obj_box'] ,
						 self.iou : blob['iou'] , self.union_box : blob['union_box'],
						 self.sub_sp : blob['sub_spatial_vector'] ,self.ob_sp:blob['ob_spatial_vector'],
						 self.rela_label: blob['rela'],self.keep_prob: 0.5,
                         self.sub_label: blob['sub_label'],self.obj_label: blob['obj_label'],
                         self.sub_obj_p: self.sub_obj_pred[blob['sub_label'], blob['obj_label']]}

            _, losses = sess.run([RD_train, self.losses], feed_dict = feed_dict)
            
            gru_loss = gru_loss + losses['gru_loss']
            vsimple_loss = vsimple_loss + losses['vsimple_loss']     
            RD_loss = RD_loss + losses['rd_loss']
            acc = acc + losses['acc']
        gru_loss = gru_loss/batch_num
        vsimple_loss = vsimple_loss/batch_num
        RD_loss = RD_loss/batch_num
        acc = acc/batch_num
        return gru_loss, vsimple_loss, RD_loss, acc

    def val_predicate(self, sess, roidb_use):
        im, im_scale = im_preprocess(roidb_use['image'])
        batch_num = len(roidb_use['index_pred'])/self.N_each_batch
        gru_loss = 0.0
        vsimple_loss = 0.0
        RD_loss = 0.0
        acc = 0.0
        for batch_id in range(np.int32(batch_num)):
            blob = get_blob_pred(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
            feed_dict = {self.image: im, self.co_occurency_log : blob['co_occurency_tensor'],
						 self.sbox: blob['sub_box'], self.obox: blob['obj_box'],
						 self.iou: blob['iou'], self.union_box: blob['union_box'],
						 self.sub_sp : blob['sub_spatial_vector'] ,self.ob_sp:blob['ob_spatial_vector'],
						 self.rela_label: blob['rela'],self.keep_prob: 1,
                         self.sub_label: blob['sub_label'],self.obj_label: blob['obj_label'],
                         self.sub_obj_p: self.sub_obj_pred[blob['sub_label'], blob['obj_label']]}

            losses = sess.run(self.losses, feed_dict=feed_dict)

            gru_loss = gru_loss + losses['gru_loss']
            vsimple_loss = vsimple_loss + losses['vsimple_loss']
            RD_loss = RD_loss + losses['rd_loss']
            acc = acc + losses['acc']
        gru_loss = gru_loss/batch_num
        vsimple_loss = vsimple_loss/batch_num
        RD_loss = RD_loss/batch_num
        acc = acc/batch_num
        return  gru_loss, vsimple_loss, RD_loss, acc

def max_pool(x, h, w, s_y, s_x, name, padding='SAME'):
    return tf.nn.max_pool(x, ksize=[1, h, w, 1], strides=[1, s_x, s_y, 1], padding=padding, name=name)
# _____________________________________________________________________________________________________#
N_cls = cfg.VRD_NUM_CLASS
N_rela = cfg.VRD_NUM_RELA
N_each_batch = cfg.VRD_BATCH_NUM
lr_init = cfg.VRD_LR_INIT
index_sp = False
index_cls = False
tf.reset_default_graph()
vnet = VSimplE()
vnet.create_graph(N_each_batch, index_sp, index_cls, N_cls, N_rela)
roidb_path = cfg.DIR + 'input/vrd_roidb.npz'
print(roidb_path)
res_path = cfg.DIR + 'Faster_RCNN/vrd_vgg_pretrained.ckpt'
print(res_path)
roidb_read = read_roidb(roidb_path)
train_roidb = roidb_read['train_roidb']
test_roidb = roidb_read['test_roidb']
N_train = len(train_roidb)
N_test = len(test_roidb)
N_round = 10
N_show = 100
N_save = N_train
N_val = N_test
total_var = tf.trainable_variables()
restore_var = [var for var in total_var if 'vgg_16' in var.name]
cls_score_var = [var for var in total_var if 'cls_score' in var.name]
res_var = [item for item in restore_var if item not in cls_score_var]
saver_res = tf.train.Saver(var_list=restore_var)
RD_var = [var for var in total_var if 'RD' in var.name]
saver = tf.train.Saver(max_to_keep=200)
for var in RD_var:
    print(var)
#___________________________________________________________________16.Adaptive Learning rate
# lr_init = 0.0001
# global_step = tf.Variable(0, trainable = False)
# learning_rate = tf.train.exponential_decay(lr_init, global_step = global_step, decay_steps = 4000, decay_rate = 0.5)
# learning_rate = tf.reduce_max([learning_rate, 0.000001])
#___________________________________________________________________Optimazer
optimizer = tf.train.AdamOptimizer(learning_rate=lr_init)
train_loss = vnet.losses['rd_loss'] #+ vnet.losses['vsimple_loss'] + vnet.losses['gru_loss']
RD_train = optimizer.minimize(train_loss, var_list = RD_var)   #, global_step = global_step

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.ConfigProto()
clip_all_weights = tf.get_collection("max_norm")

with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(clip_all_weights)
    saver_res.restore(sess, res_path)
    Epoch = 0
    gru_loss = 0.0
    vsimple_loss = 0.0
    rd_loss = 0.0
    acc = 0.0
    for Epoch in range(2):
        t=0    
        for r in range(N_round):
            for roidb_id in range(N_train):
                roidb_use = train_roidb[roidb_id]
                if len(roidb_use['rela_gt']) == 0:
                    continue
                gru_loss_temp, vsimple_loss_temp, rd_loss_temp, acc_temp = vnet.train_predicate(sess,roidb_use,RD_train)
                gru_loss = gru_loss + gru_loss_temp
                vsimple_loss = vsimple_loss + vsimple_loss_temp
                rd_loss = rd_loss + rd_loss_temp
                acc = acc + acc_temp
                t = t + 1
                if t % N_show == 0:
                    #lr = sess.run(learning_rate)
                    print("Epoch:{0}__ T:{1}>>> GRU : {2}| VSimplE : {3} | RD: {4} ___ Acc : {5} ".format(Epoch+1,t,round(gru_loss/N_show,2),round(vsimple_loss/N_show,2), round(rd_loss/N_show, 2),round(acc, 2)))
                    gru_loss = 0.0
                    vsimple_loss = 0.0
                    rd_loss = 0.0
                    acc = 0.0

                if t % N_save == 0:				
                    save_path = cfg.DIR + 'Model_ckpt/Pred' + format(int(t / N_save), '04') + '.ckpt'
                    saver.save(sess, save_path)
                    print("Model Saved to {0}".format(save_path))
                    gru_loss_val = 0.0
                    vsimple_loss_val = 0.0
                    rd_loss_val = 0.0
                    acc_val = 0.0
                    for val_id in range(N_val):
                        roidb_use = test_roidb[val_id]
                        if len(roidb_use['rela_gt']) == 0:
                            continue
                        gru_loss_temp, vsimple_loss_temp, rd_loss_temp, acc_temp = vnet.val_predicate(sess,roidb_use)

                        gru_loss_val = gru_loss + gru_loss_temp
                        vsimple_loss_val = vsimple_loss + vsimple_loss_temp
                        rd_loss_val = rd_loss_val + rd_loss_temp
                        acc_val = acc_val + acc_temp
                    print("\n************************************GRU_Loss : {0} , VSimplE_Loss : {1} ,Total_Loss: {2} ____ Acc: {3}\n".format(round(gru_loss_val / N_val, 2), round(vsimple_loss_val / N_val, 2), round(rd_loss_val / N_val, 2), round(100 * acc_val/ N_val, 4)))
print(" \nDone!    Distance to best result ", round((100 * acc_val / N_val) - 60), 2)